# 🎯 Projenin Amacı

Bu çalışmanın temel amacı, sosyal medya platformlarından biri olan Twitter üzerinde paylaşılan gönderiler aracılığıyla kullanıcıların duygu durumlarını sınıflandırmaktır. Doğal dil işleme (NLP) teknikleri ve makine öğrenmesi algoritmaları kullanılarak, tweet metinleri üzerinden otomatik duygu analizi gerçekleştirilmiştir. Bu analiz sonucunda, her bir tweet’in olumlu (positive), olumsuz (negative) veya nötr (neutral) olarak sınıflandırılması hedeflenmiştir.

### 📦 1. Gerekli Kütüphanelerin Yüklenmesi ve Verinin İçeri Aktarılması

Bu bölümde, proje için gerekli olan Python kütüphaneleri yüklenir ve CSV dosyasından veri seti içeri alınır.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk

# Veriyi yükle
data = pd.read_csv("twitter.csv")
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


### 🧽 2. Metin Temizleme İşlemleri

Bu bölümde, tweet metinleri temizlenerek makine öğrenmesi için daha uygun hale getirilir.

In [2]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string

stopword = set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

# Temizleme işlemini uygulama
data["tweet"] = data["tweet"].apply(clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AhmetD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 🧠 3. Duygu Analizi Skorlarının Hesaplanması

Tweet'ler üzerinden pozitif, negatif ve nötr duygu skorları hesaplanır ve veri setine eklenir.

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sentiments = SentimentIntensityAnalyzer()

data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["tweet"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["tweet"]]
data["Neutral"]  = [sentiments.polarity_scores(i)["neu"] for i in data["tweet"]]

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\AhmetD\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### 🧾 4. Kullanılacak Sütunların Seçilmesi

Yalnızca analiz için gerekli sütunlar (tweet, pozitif, negatif, nötr) tutulur.

In [4]:
data = data[["tweet", "Positive", "Negative", "Neutral"]]
print(data.head())

                                               tweet  Positive  Negative  \
0   rt mayasolov woman shouldnt complain clean ho...     0.147     0.157   
1   rt  boy dat coldtyga dwn bad cuffin dat hoe  ...     0.000     0.280   
2   rt urkindofbrand dawg rt  ever fuck bitch sta...     0.000     0.577   
3             rt cganderson vivabas look like tranni     0.333     0.000   
4   rt shenikarobert shit hear might true might f...     0.154     0.407   

   Neutral  
0    0.696  
1    0.720  
2    0.423  
3    0.667  
4    0.440  


### 📊 5. Genel Duygu Durumunun Belirlenmesi

Tüm tweet'ler için toplam duygu puanları hesaplanır ve hangi duygu öne çıkıyorsa o duygu durumunu temsil eder.

In [5]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

def sentiment_score(a, b, c):
    if (a > b) and (a > c):
        print("Positive 😊 ")
    elif (b > a) and (b > c):
        print("Negative 😠 ")
    else:
        print("Neutral 🙂 ")

sentiment_score(x, y, z)

Neutral 🙂 


### 📈 6. Duygu Skorlarının Sayısal Olarak Görüntülenmesi

Toplam pozitif, negatif ve nötr duygu puanları terminale yazdırılır.

In [6]:
print("Positive: ", x)
print("Negative: ", y)
print("Neutral: ", z)

Positive:  2880.086000000009
Negative:  7201.020999999922
Neutral:  14696.887999999733
